In [10]:
import torch
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms



In [4]:


class CustomDataSet(Dataset):
    def __init__(self, filepath,test=False,transform=None):
        self.data = pd.read_csv(filepath)
        self.transform = transform
        self.test = test
    def __getitem__(self, index):
        if self.test ==True:

            #W X H X C
            image = self.data.iloc[index,0:].values.astype(np.uint8).reshape(28,28,1)
            label = 0 #dummy labe
        else:
            image = self.data.iloc[index,1:].values.astype(np.uint8).reshape(28,28,1)
            label = self.data.iloc[index,0]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.data)

In [9]:
mean, std = 0.5,0.5

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean, std)])

trainDataSet = CustomDataSet('digit-recognizer/train.csv',transform = transform)
trainLoader = DataLoader(trainDataSet, batch_size=64, shuffle=True)

testDataSet = CustomDataSet('digit-recognizer/test.csv',transform = transform,test=True)
testLoader = DataLoader(testDataSet, batch_size=28000, shuffle=False)


In [16]:
## Creating NN Model

class NET(nn.Module):
    def __init__(self):
        super(NET, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = self.drop(self.conv2(x))
        x = F.relu(F.max_pool2d(x,2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
device = torch.device('mps')

model = NET().to(device)

In [32]:
## Configure Loss and Optimizer
# Loss and optimizer
criterion = nn.NLLLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [33]:
## Training Loop


NUM_EPOCHS = 10



for epoch in range(NUM_EPOCHS):

    cum_loss = 0
    print(f"Epoch {epoch+1} of {NUM_EPOCHS}")
    for batch in trainLoader:
        ## move batch to gpu
        optimizer.zero_grad()


        image, labels = batch
        image = image.to(device)
        labels = labels.to(device)

        ## forward pass 
        Y = model(image)
        loss = criterion(Y,labels)

        ## backward pass
        loss.backward()
        optimizer.step()
        cum_loss += loss.item()

    print("Cumulative Loss: {}".format(cum_loss/len(trainLoader)))




Epoch 1 of 10
Cumulative Loss: 0.10302404367862499
Epoch 2 of 10
Cumulative Loss: 0.09589839305713081
Epoch 3 of 10
Cumulative Loss: 0.09149323989429173
Epoch 4 of 10
Cumulative Loss: 0.09084535211245309
Epoch 5 of 10
Cumulative Loss: 0.08541024566870438
Epoch 6 of 10
Cumulative Loss: 0.0861640335499207
Epoch 7 of 10
Cumulative Loss: 0.08335387028101336
Epoch 8 of 10
Cumulative Loss: 0.08328832598910575
Epoch 9 of 10
Cumulative Loss: 0.07892929465733922
Epoch 10 of 10
Cumulative Loss: 0.0807679772739018
